<a href="https://colab.research.google.com/github/graphlit/graphlit-samples/blob/main/python/Notebook%20Examples/Graphlit_2025_02_19_Analyze_Trello_Feed_Ingestion_Costs_%26_Usage.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Description**

This example shows how to analyze the credit usage and individual usage logs from a Graphlit preparation workflow applied to a Trello feed.

**Requirements**

Prior to running this notebook, you will need to [signup](https://docs.graphlit.dev/getting-started/signup) for Graphlit, and [create a project](https://docs.graphlit.dev/getting-started/create-project).

You will need the Graphlit organization ID, preview environment ID and JWT secret from your created project.

Assign these properties as Colab secrets: GRAPHLIT_ORGANIZATION_ID, GRAPHLIT_ENVIRONMENT_ID and GRAPHLIT_JWT_SECRET.

To access your Trello account, assign these properties as Colab secrets: TRELLO_API_KEY, TRELLO_API_TOKEN, and TRELLO_BOARD_IDENTIFIER.

---

Install Graphlit Python client SDK

In [ ]:
!pip install --upgrade graphlit-client

In [ ]:
!pip install --upgrade isodate

In [ ]:
!pip install --upgrade tiktoken

Initialize Graphlit

In [ ]:
import os
from google.colab import userdata
from graphlit import Graphlit
from graphlit_api import input_types, enums, exceptions

os.environ['GRAPHLIT_ORGANIZATION_ID'] = userdata.get('GRAPHLIT_ORGANIZATION_ID')
os.environ['GRAPHLIT_ENVIRONMENT_ID'] = userdata.get('GRAPHLIT_ENVIRONMENT_ID')
os.environ['GRAPHLIT_JWT_SECRET'] = userdata.get('GRAPHLIT_JWT_SECRET')

graphlit = Graphlit()

Initialize Trello credentials

In [ ]:
os.environ['TRELLO_API_KEY'] = userdata.get('TRELLO_API_KEY')
os.environ['TRELLO_API_TOKEN'] = userdata.get('TRELLO_API_TOKEN')

os.environ['TRELLO_BOARD_IDENTIFIER'] = userdata.get('TRELLO_BOARD_IDENTIFIER')

Define Graphlit helper functions

In [ ]:
from typing import List, Optional
from datetime import datetime, timedelta
import isodate

async def create_feed(correlation_id: str):
    if graphlit.client is None:
        return;

    input = input_types.FeedInput(
        name="Trello",
        type=enums.FeedTypes.ISSUE,
        issue=input_types.IssueFeedPropertiesInput(
            type=enums.FeedServiceTypes.TRELLO,
            trello=input_types.TrelloFeedPropertiesInput(
                key=os.environ['TRELLO_API_KEY'],
                token=os.environ['TRELLO_API_TOKEN'],
                type=enums.TrelloTypes.BOARD,
                identifiers=[os.environ['TRELLO_BOARD_IDENTIFIER']],
            ),
            readLimit=10 # limiting to 10 issues
        )
    )

    try:
        response = await graphlit.client.create_feed(input, correlation_id=correlation_id)

        return response.create_feed.id if response.create_feed is not None else None
    except exceptions.GraphQLClientError as e:
        print(str(e))
        return None

    return None

async def is_feed_done(feed_id: str):
    if graphlit.client is None:
        return;

    response = await graphlit.client.is_feed_done(feed_id)

    return response.is_feed_done.result if response.is_feed_done is not None else None

async def query_contents(feed_id: str):
    if graphlit.client is None:
        return;

    try:
        response = await graphlit.client.query_contents(
            filter=input_types.ContentFilter(
                feeds=[
                    input_types.EntityReferenceFilter(
                        id=feed_id
                    )
                ]
            )
        )

        return response.contents.results if response.contents is not None else None
    except exceptions.GraphQLClientError as e:
        print(str(e))
        return None

async def lookup_usage(correlation_id: str):
    if graphlit.client is None:
        return;

    try:
        response = await graphlit.client.lookup_usage(correlation_id)

        return response.lookup_usage if response.lookup_usage is not None else None
    except exceptions.GraphQLClientError as e:
        print(str(e))
        return None

async def lookup_credits(correlation_id: str):
    if graphlit.client is None:
        return;

    try:
        response = await graphlit.client.lookup_credits(correlation_id)

        return response.lookup_credits if response.lookup_credits is not None else None
    except exceptions.GraphQLClientError as e:
        print(str(e))
        return None

async def delete_all_feeds():
    if graphlit.client is None:
        return;

    _ = await graphlit.client.delete_all_feeds(is_synchronous=True)

def dump_usage_record(record):
    print(f"{record.date}: {record.name}")

    duration = isodate.parse_duration(record.duration)

    if record.workflow:
        print(f"- Workflow [{record.workflow}] took {duration}, used credits [{record.credits:.8f}]")
    else:
        print(f"- Operation took {duration}, used credits [{record.credits:.8f}]")

    if record.entity_id:
        if record.entity_type:
            if record.entity_type == enums.EntityTypes.CONTENT and record.content_type:
                print(f"- {record.entity_type} [{record.entity_id}]: Content type [{record.content_type}], file type [{record.file_type}]")
            else:
                print(f"- {record.entity_type} [{record.entity_id}]")
        else:
            print(f"- Entity [{record.entity_id}]")

    if record.model_service:
        print(f"- Model service [{record.model_service}], model name [{record.model_name}]")

    if record.processor_name:
        if record.processor_name in ["Deepgram Audio Transcription", "Assembly.AI Audio Transcription"]:
            length = timedelta(milliseconds=record.count or 0)

            if record.model_name:
                print(f"- Processor name [{record.processor_name}], model name [{record.model_name}], length [{length}]")
            else:
                print(f"- Processor name [{record.processor_name}], length [{length}]")
        else:
            if record.count:
                if record.model_name:
                    print(f"- Processor name [{record.processor_name}], model name [{record.model_name}], units [{record.count}]")
                else:
                    print(f"- Processor name [{record.processor_name}], units [{record.count}]")
            else:
                if record.model_name:
                    print(f"- Processor name [{record.processor_name}], model name [{record.model_name}]")
                else:
                    print(f"- Processor name [{record.processor_name}]")

    if record.uri:
        print(f"- URI [{record.uri}]")

    if record.name == "Prompt completion":
        if record.prompt:
            print(f"- Prompt [{record.prompt_tokens} tokens (includes RAG context tokens)]:")
            print(record.prompt)

        if record.completion:
            print(f"- Completion [{record.completion_tokens} tokens (includes JSON guardrails tokens)], throughput: {record.throughput:.3f} tokens/sec:")
            print(record.completion)

    elif record.name == "Text embedding":
        if record.prompt_tokens is not None:
            print(f"- Text embedding [{record.prompt_tokens} tokens], throughput: {record.throughput:.3f} tokens/sec")

    elif record.name == "Document preparation":
        if record.prompt_tokens is not None and record.completion_tokens is not None:
            print(f"- Document preparation [{record.prompt_tokens} input tokens, {record.completion_tokens} output tokens], throughput: {record.throughput:.3f} tokens/sec")

    elif record.name == "Data extraction":
        if record.prompt_tokens is not None and record.completion_tokens is not None:
            print(f"- Data extraction [{record.prompt_tokens} input tokens, {record.completion_tokens} output tokens], throughput: {record.throughput:.3f} tokens/sec")

    elif record.name == "GraphQL":
        if record.request:
            print(f"- Request:")
            print(record.request)

        if record.variables:
            print(f"- Variables:")
            print(record.variables)

        if record.response:
            print(f"- Response:")
            print(record.response)

    if record.name.startswith("Upload"):
        print(f"- File upload [{record.count} bytes], throughput: {record.throughput:.3f} bytes/sec")

    print()


In [ ]:
import tiktoken

model = "gpt-4o"
encoding = tiktoken.encoding_for_model(model)

In [ ]:
import json
import requests

def parse_token_count(uri):
    try:
        response = requests.get(uri)
        response.raise_for_status()  # Raise an error for HTTP issues

        data = response.json()

        tok_value = data.get("tok")

        return tok_value

    except requests.exceptions.RequestException as e:
        print(f"Error fetching the JSON file: {e}")
    except json.JSONDecodeError as e:
        print(f"Error decoding JSON: {e}")

    return None


Execute Graphlit example

In [ ]:
from IPython.display import display, Markdown, HTML
import time
from datetime import datetime

# Remove any existing feeds; only needed for notebook example
await delete_all_feeds()

print('Deleted all feeds.')

# NOTE: create a unique cost correlation ID
correlation_id = datetime.now().isoformat()

feed_id = await create_feed(correlation_id=correlation_id)

if feed_id is not None:
    print(f'Created feed [{feed_id}].')

    # Wait for feed to complete, since ingestion happens asychronously
    done = False
    time.sleep(5)
    while not done:
        done = await is_feed_done(feed_id)

        if not done:
            time.sleep(10)

    print(f'Completed feed [{feed_id}].')


In [ ]:
    # Query contents by feed
    contents = await query_contents(feed_id)

    if contents is not None:
        print(f'Found {len(contents)} contents in feed [{feed_id}].')

        for content in contents:
            if content is not None:

                print(f'Ingested content [{content.id}], state [{content.state}]:')

                if content.markdown is not None:
                    print(content.markdown)

                print()

In [ ]:
from IPython.display import display, HTML, JSON

time.sleep(10) # give it some time for billing events to catch up

credits = await lookup_credits(correlation_id)

if credits is not None:
    display(Markdown(f"### Credits used: {credits.credits:.6f}"))
    print(f"- storage [{credits.storage_ratio:.2f}%], compute [{credits.compute_ratio:.2f}%]")
    print(f"- embedding [{credits.embedding_ratio:.2f}%], completion [{credits.completion_ratio:.2f}%]")
    print(f"- ingestion [{credits.ingestion_ratio:.2f}%], indexing [{credits.indexing_ratio:.2f}%], preparation [{credits.preparation_ratio:.2f}%], extraction [{credits.extraction_ratio:.2f}%], enrichment [{credits.enrichment_ratio:.2f}%], publishing [{credits.publishing_ratio:.2f}%]")
    print(f"- search [{credits.search_ratio:.2f}%], conversation [{credits.conversation_ratio:.2f}%]")
    print()

usage = await lookup_usage(correlation_id)

if usage is not None:
    display(Markdown(f"### Usage records:"))

    for record in usage:
        dump_usage_record(record)
    print()
